# Упражнение 03 : Агрегации

In [13]:
import pandas as pd
import sqlite3

## создайте подключение к базе данных с помощью библиотеки sqlite3

In [14]:
con = sqlite3.connect('../data/checking-logs.sqlite')

## получаем схему таблицы test

In [15]:
sql = 'PRAGMA table_info(test)'
df = pd.read_sql(sql, con)
df

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## получите только первые 10 строк таблицы, чтобы проверить, как она выглядит

In [16]:
sql = 'SELECT * FROM test LIMIT 10'
df = pd.read_sql(sql, con)
df

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## найдите среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком выполнения соответствующей задачи, используя только один запрос
* сделайте это, соединив таблицу с таблицей крайних сроков
* разница должна отображаться в часах
* не принимайте во внимание лабораторный проект 1, у него более длительные сроки и он будет выделяться
* значение должно быть сохранено во фрейме данных df_min с соответствующим uid

In [17]:
sql = '''
SELECT
    test.uid,
    MIN((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS df_min
FROM test
JOIN deadlines
ON test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
'''
df_min = pd.read_sql(sql, con)
df_min

,uid,df_min
0,user_30,-202


## сделайте то же самое, но для максимума, используя только один запрос, имя фрейма данных — df_max

In [18]:
sql = '''
SELECT
    test.uid,
    MAX((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS df_min
FROM test
JOIN deadlines
ON test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
'''
df_max = pd.read_sql(sql, con)
df_max

,uid,df_min
0,user_25,-2


## сделайте то же самое, но для среднего значения, используя только один запрос. На этот раз ваш фрейм данных не должен содержать столбец uid, а имя фрейма данных — df_avg

In [19]:
sql = '''
SELECT
    AVG((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS df_avg
FROM test
JOIN deadlines
ON test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
'''
df_avg = pd.read_sql(sql, con)
df_avg

,df_avg
0,-89.125


## мы хотим проверить гипотезу о том, что у пользователей, которые просматривали ленту новостей всего несколько раз, разница между первым просмотром и дедлайном меньше. Для этого нужно вычислить коэффициент корреляции между количеством просмотров и разницей
* используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews
* uid - это жидкости, которые существуют в тесте
* avg_diff — это средняя разница между первым коммитом и крайним сроком выполнения задания для каждого пользователя
* просмотры страниц - это количество посещений ленты новостей на одного пользователя
* не принимайте во внимание лабораторный ’проект 1’
* сохраните его в фрейме данных views_diff
* Используйте метод Pandas corr() для вычисления коэффициента корреляции между количеством просмотров страниц и разницей

In [20]:
sql = '''
SELECT
    uid,
    AVG((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_diff,
    COUNT(uid) as pageviews
FROM test
JOIN deadlines
ON test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY uid;
'''
views_diff = pd.read_sql(sql, con)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,5
1,user_10,-74.800000,5
2,user_14,-159.000000,3
3,user_17,-61.600000,5
4,user_18,-5.666667,3
5,user_19,-98.750000,4
6,user_21,-95.500000,4
7,user_25,-92.600000,5
8,user_28,-86.400000,5
9,user_3,-105.400000,5


In [21]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,0.117685
pageviews,0.117685,1.000000


## закройте соединение

In [22]:
con.close()